# Imports & Data Load

In [63]:
# imports   
import pandas as pd
import numpy as np
import requests
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_columns', None)

In [2]:
# download data

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
# create dataframe

df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])
df

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...
...,...,...,...,...
943,mlops-zoomcamp,Module 6: Best practices,Github actions: Permission denied error when e...,Problem description\nThis is the step in the c...
944,mlops-zoomcamp,Module 6: Best practices,Managing Multiple Docker Containers with docke...,Problem description\nWhen a docker-compose fil...
945,mlops-zoomcamp,Module 6: Best practices,AWS regions need to match docker-compose,Problem description\nIf you are having problem...
946,mlops-zoomcamp,Module 6: Best practices,Isort Pre-commit,Problem description\nPre-commit command was fa...


In [5]:
# preview data
df.sample(5)

,course,section,question,text
143,data-engineering-zoomcamp,Module 1: Docker and Terraform,"Terraform - Error:Post ""https://storage.google...",The issue was with the network. Google is not ...
186,data-engineering-zoomcamp,Module 2: Workflow Orchestration,"Since I was using slow laptop, and we have so ...","CSV Files are very big in nyc data, so we inst..."
882,mlops-zoomcamp,Module 2: Experiment tracking,Viewing SQLlite Data Raw & Deleting Experiment...,All the experiment and other tracking informat...
155,data-engineering-zoomcamp,Module 1: Docker and Terraform,SQL - SELECT Zone FROM taxi_zones Error Column...,It is inconvenient to use quotation marks all ...
475,machine-learning-zoomcamp,1. Introduction to Machine Learning,How to identify the shape of dataset in Pandas,There are many ways to identify the shape of d...


In [6]:
# filter dataset
df[df['course']=='data-engineering-zoomcamp']

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...
...,...,...,...,...
430,data-engineering-zoomcamp,Workshop 2 - RisingWave,Unable to Open Dashboard as xdg-open doesn’t o...,Refer to the solution given in the first solut...
431,data-engineering-zoomcamp,Workshop 2 - RisingWave,Resolving Python Interpreter Path Inconsistenc...,Example Error:\nWhen attempting to execute a P...
432,data-engineering-zoomcamp,Workshop 2 - RisingWave,How does windowing work in Sql?,Ans : Windowing in streaming SQL involves defi...
433,data-engineering-zoomcamp,Triggers in Mage via CLI,"Encountering the error ""ModuleNotFoundError: N...","Python 3.12.1, is not compatible with kafka-py..."


# Implementing Basic Search

Vector Spaces
- Turn documents into vectors
- Term-document matrix
    - rows: documents
    - columns: words / tokens
- bag of words
    - we do not care about the order of tokens
    - we only care about the occurence of tokens
    - sparse matrix

## Bag of Words with `CountVectorizer`

### Example Implementation

In [23]:
# example data
sample_documents = [
    "Course starts on 15th Jan 2024",
    "Prerequisites listed on GitHub",
    "Submit homeworks after start date",
    "Registration not required for participation",
    "Setup Google Cloud and Python before course"
]

In [38]:
cv = CountVectorizer(stop_words='english')
cv.fit(sample_documents)

CountVectorizer(stop_words='english')

In [39]:
cv.get_feature_names_out()  # different tokens in our dataset

array(['15th', '2024', 'cloud', 'course', 'date', 'github', 'google',
       'homeworks', 'jan', 'listed', 'participation', 'prerequisites',
       'python', 'registration', 'required', 'setup', 'start', 'starts',
       'submit'], dtype=object)

In [40]:
X = cv.transform(sample_documents)
X

<5x19 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [41]:
# condense sparse matrix for visualization
pd.DataFrame(X.todense(), columns=cv.get_feature_names_out(), index=sample_documents).T

,Course starts on 15th Jan 2024,Prerequisites listed on GitHub,Submit homeworks after start date,Registration not required for participation,Setup Google Cloud and Python before course
15th,1,0,0,0,0
2024,1,0,0,0,0
cloud,0,0,0,0,1
course,1,0,0,0,1
date,0,0,1,0,0
github,0,1,0,0,0
google,0,0,0,0,1
homeworks,0,0,1,0,0
jan,1,0,0,0,0
listed,0,1,0,0,0


### Implementation on Dataset

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english', min_df=5)
X = cv.fit_transform(df['text'])

names = cv.get_feature_names_out()

# df_docs = pd.DataFrame(X.toarray(), columns=names, index=df['text']).T
df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947
01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

- more the word occurs, less important or significant it is
- to understand that, we use TF-IDF

## TF-IDF

In [50]:
cv = TfidfVectorizer(stop_words='english', min_df=5)
X = cv.fit_transform(df['text'])
names = cv.get_feature_names_out()
tfidf_docs = pd.DataFrame(X.toarray(), columns=names).T.round(2)
tfidf_docs

0     1     2    3    4     5     6    7     8     9    10   11   \
01        0.0  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.0   
02        0.0  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.0   
03        0.0  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.0   
04        0.0  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.0   
05        0.0  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.0   
...       ...   ...   ...  ...  ...   ...   ...  ...   ...   ...  ...  ...   
yes       0.0  0.00  0.28  0.0  0.0  0.00  0.21  0.2  0.15  0.00  0.0  0.0   
yml       0.0  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.0   
youtube   0.0  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.00  0.15  0.0  0.0   
zip       0.0  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.0   
zoomcamp  0.0  0.43  0.00  0.0  0.0  0.19  0.00  0.0  0.00  0.00  0.0  0.0   

           12    13   14   15    16   17    18    19   20   21   22   23   \
01        0.00  0.00  0.0  0.0  0.00  0.0  0.00  0.00  0.0  0.0  0.0  0.0   
02        0.00  0.00  0.0  0.0  0.00  0.0  0.00  0.00  0.0  0.0  0.0  0.0   
03        0.00  0.00  0.0  0.0  0.00  0.0  0.00  0.00  0.0  0.0  0.0  0.0   
04        0.00  0.00  0.0  0.0  0.00  0.0  0.00  0.00  0.0  0.0  0.0  0.0   
05        0.00  0.00  0.0  0.0  0.00  0.0  0.00  0.00  0.0  0.0  0.0  0.0   
...        ...   ...  ...  ...   ...  ...   ...   ...  ...  ...  ...  ...   
yes       0.00  0.26  0.0  0.0  0.25  0.0  0.00  0.32  0.0  0.0  0.0  0.0   
yml       0.00  0.00  0.0  0.0  0.00  0.0  0.00  0.00  0.0  0.0  0.0  0.0   
youtube   0.33  0.00  0.0  0.0  0.00  0.0  0.00  0.00  0.0  0.0  0.0  0.0   
zip       0.00  0.00  0.0  0.0  0.00  0.0  0.00  0.00  0.0  0.0  0.0  0.0   
zoomcamp  0.00  0.00  0.0  0.0  0.00  0.0  0.15  0.00  0.0  0.0  0.0  0.0   

          24   25   26   27    28   29   30   31    32    33   34    35   \
01        0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.0  0.00   
02        0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.0  0.00   
03        0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.0  0.00   
04        0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.0  0.00   
05        0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.0  0.00   
...       ...  ...  ...  ...   ...  ...  ...  ...   ...   ...  ...   ...   
yes       0.0  0.0  0.0  0.0  0.16  0.0  0.0  0.0  0.47  0.12  0.0  0.37   
yml       0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.0  0.00   
youtube   0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.0  0.00   
zip       0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.0  0.00   
zoomcamp  0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.0  0.00   

           36    37   38   39   40   41   42   43   44    45   46   47    48   \
01        0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.08  0.0  0.0  0.00   
02        0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.00   
03        0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.00   
04        0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.00   
05        0.19  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.00   
...        ...   ...  ...  ...  ...  ...  ...  ...  ...   ...  ...  ...   ...   
yes       0.00  0.25  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.00   
yml       0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.00   
youtube   0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.00   
zip       0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.25   
zoomcamp  0.00  0.22  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.00   

          49   50   51   52   53    54   55   56   57   58   59   60   61   \
01        0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
02        0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
03        0.0  0.0  0.0  0.0

- more important the word is, higher is its weight

In [51]:
# search on user query
query = "Do I need to know python to sign up for the January course?"
q = cv.transform([query])
q.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [61]:
# let's compute the similarity between documents and the query
X.dot(q.T).todense()   # similarities for each document

matrix([[0.19464486],
        [0.        ],
        [0.        ],
        [0.06011641],
        [0.04932915],
        [0.        ],
        [0.        ],
        [0.13477565],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.15899187],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.07431408],
        [0.        ],
        [0.        ],
        [0.05779673],
        [0.07243428],
        [0.        ],
        [0.05174293],
        [0.16373635],
        [0.08076031],
        [0.        ],
        [0.09755254],
        [0.        ],
        [0.21069625],
        [0.12067781],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.06381749],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.00910541],
        [0.02835681],
        [0.05480112],
        [0.        ],
        [0.        ],
        [0.        ],
        [0

- here by calculating the dot product between the vectors, we have calculated the COSINE SIMILARITY

In [57]:
cosine_similarity(X, q)

array([[0.19464486],
       [0.        ],
       [0.        ],
       [0.06011641],
       [0.04932915],
       [0.        ],
       [0.        ],
       [0.13477565],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.15899187],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.07431408],
       [0.        ],
       [0.        ],
       [0.05779673],
       [0.07243428],
       [0.        ],
       [0.05174293],
       [0.16373635],
       [0.08076031],
       [0.        ],
       [0.09755254],
       [0.        ],
       [0.21069625],
       [0.12067781],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.06381749],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.00910541],
       [0.02835681],
       [0.05480112],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.024

convert the result into a vector

In [75]:
score = cosine_similarity(X, q).flatten()
np.argsort(score)   # returns indices of the documents in lowest to highest score order

array([764,  27, 806, 577, 445], dtype=int64)

In [77]:
# top 5 highest scoring documents
np.argsort(score)[-5:]

array([764,  27, 806, 577, 445], dtype=int64)

In [82]:
df

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...
...,...,...,...,...
943,mlops-zoomcamp,Module 6: Best practices,Github actions: Permission denied error when e...,Problem description\nThis is the step in the c...
944,mlops-zoomcamp,Module 6: Best practices,Managing Multiple Docker Containers with docke...,Problem description\nWhen a docker-compose fil...
945,mlops-zoomcamp,Module 6: Best practices,AWS regions need to match docker-compose,Problem description\nIf you are having problem...
946,mlops-zoomcamp,Module 6: Best practices,Isort Pre-commit,Problem description\nPre-commit command was fa...


In [84]:
print("> User Query:", query)
print("> Closest Question:\n", df['question'][445])
print("> Match Score:", score[445])
print("> Response:\n", df['text'][445])

> User Query: Do I need to know python to sign up for the January course?
> Closest Question:
 How much Python should I know?
> Match Score: 0.36726644770526745
> Response:
 Check this article. If you know everything in this article, you know enough. If you don’t, read the article and join the coursIntroduction to Pythone too :)
Introduction to Python – Machine Learning Bookcamp
You can follow this English course from the OpenClassrooms e-learning platform, which is free and covers the python basics for data analysis: Learn Python Basics for Data Analysis - OpenClassrooms . It is important to know some basics such as: how to run a Jupyter notebook, how to import libraries (and what libraries are), how to declare a variable (and what variables are) and some important operations regarding data analysis.
(Mélanie Fouesnard)


In [85]:
match = np.argsort(score)[-1]

445

In [98]:
def calculate_similarity(query:str):
    q = cv.transform([query])
    q.toarray()
    score = cosine_similarity(X, q).flatten()
    # top_5 = np.argsort(score)[-5:]
    print("> User Query:", query)
    print("- - -")
    match_1 = np.argsort(score)[-1]
    print("> Closest Question:\n", df['question'][match_1])
    print("> Match Score:", score[match_1].round(3))
    print("> Response:\n", df['text'][match_1])
    print("- - -")
    match_2 = np.argsort(score)[-2]
    print("> Closest Question:\n", df['question'][match_2])
    print("> Match Score:", score[match_2].round(3))
    print("> Response:\n", df['text'][match_2])
    print("- - -")
    match_3 = np.argsort(score)[-3]
    print("> Closest Question:\n", df['question'][match_3])
    print("> Match Score:", score[match_3].round(3))
    print("> Response:\n", df['text'][match_3])

In [99]:
query="I just discovered the course, is it too late to join?"
calculate_similarity(query=query)

> User Query: I just discovered the course, is it too late to join?
- - -
> Closest Question:
 Course - When will the course start?
> Match Score: 0.48
> Response:
 The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.
- - -
> Closest Question:
 I filled the form, but haven't received a confirmation email. Is it normal?
> Match Score: 0.357
> Response:
 The process is automated now, so you should receive the email eventually. If you haven’t, check your promotions tab in Gmail as well as spam.
If you unsubscribed from our newsletter, you won't get course related updates too.
But don't wor

# END